In [92]:
import threading
import datetime

a = [[5,"work1"],[10,"work2"],[3,"work3"]]
c = 0
work_scheduler = []

def to_do():
    print("{:%H:%M:%S}".format(datetime.datetime.now()) + " do " + work_scheduler[0])
    work_scheduler.pop(0)

print("{:%H:%M:%S}".format(datetime.datetime.now()) + " start")

for i in sorted(a):
    work_scheduler.append(i[c+1])
    t = threading.Timer(i[c], to_do)
    t.start()

17:16:49 start
17:16:52 do work3
17:16:54 do work1
17:16:59 do work2
